In [1]:
def kakao_restaurant(station, cnt):  # 15의 배수로 입력(15,30,....,150...)
    import requests

    app_key = "f58ecd8a6f3a91c7b483596610cb46f9"
    headers = {"Authorization": "KakaoAK {}".format(app_key)}
    query = station + "맛집"
    pages = range(cnt // 15)  # 1페이지당 15개의 음식점의 정보를 불러올 수 있다.
    url = (
        "https://dapi.kakao.com/v2/local/search/keyword.json?query={}&pages={}".format(
            query, pages
        )
    )
    urls = []
    for page in pages:
        response = requests.get(url, headers=headers)
        for dic in response.json()["documents"]:
            urls.append(dic["place_url"])
    return urls

from selenium import webdriver

from selenium.webdriver.common.by import By
import time


def kakao_inform_restaurant(url):
    options = webdriver.ChromeOptions()
    # 창 숨기는 옵션 추가
    options.add_argument("headless")
    driver = webdriver.Chrome(
        "chromedriver.exe",
        options=options,
    )
    # url  url 리스트 안의 원소들
    driver.get(url)  # 일단 하나의 사이트에서만
    driver.implicitly_wait(10)

    name = driver.find_element(
        by=By.XPATH, value="/html/body/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/div/h2"
    )
    restaurant_name = name.text  # 음식점 명
    address = driver.find_element(
        by=By.XPATH,
        value="/html/body/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/div/span[1]",
    )
    address_name = address.text
    tag = driver.find_element(
        by=By.XPATH,
        value="/html/body/div[2]/div[2]/div[2]/div[1]/div[1]/div[2]/div/div",
    )  # 평점 정보
    try:
        
        sort_point, cnt, review = tag.text.split("\n")
        sort, _, point = sort_point.split()  # 분류,평점
        point = float(point)
        cnt = int(cnt.strip(")").strip("(").replace(",", ""))  # 평점을 매긴 사람들의 수
        review_count = int(review.split()[1].replace(",", ""))  # 리뷰 수
    except:
        
        
       

        
        sort = tag.text.split(' ')[0]
        point= 0
        cnt =0  # 평점을 매긴 사람들의 수
        review_count =0 # 리뷰 수
    try:
        try:

            menus = driver.find_element(
                by=By.XPATH, value="/html/body/div[2]/div[2]/div[2]/div[3]/ul"
            )

        except:
            pass

        ssum = 0
        menu_count = 0
        menu_list = menus.text.split("\n")[::2]  # 대표 메뉴 리스트

        for i in menus.text.split("\n")[1::2]:
            try:
                ssum += int(i.replace(",", ""))
                menu_count += 1
            except:
                continue
        mean_price = ssum // menu_count  # 평균 가격대
    except:
        menu_list = ["표기안함"]
        mean_price = "표기안함"
        pass
    # 여기서부턴 리뷰들

    review_list = []
    daum_button, no_review = (
        False,
        False,
    )  # 처음에는 2페이지로 가는 파라미터가 1이지만, 한번 다음을 클릭한다면, 2로 바뀐다
    next_page = 1
    while 1:
        if len(review_list) >= 30:
            break
        for n in range(1, 5 + 1):
            try:
                review_coment = driver.find_element(
                    by=By.XPATH,
                    value=f"/html/body/div[2]/div[2]/div[2]/div[6]/div[3]/ul/li[{n}]/div[2]/p",
                ).text
                if review_coment != "":
                    review_list.append(review_coment)

            except:
                no_review = True
                break
        if no_review:
            break
        try:

            next_page_button = driver.find_element(
                by=By.XPATH,
                value=f"/html/body/div[2]/div[2]/div[2]/div[6]/div[3]/div/a[{next_page}]",
            )
            next_page_button.click()
            time.sleep(3)
            next_page += 1
        except:
            pass
        try:
            if next_page == 6:
                next_page_button = driver.find_element(
                    by=By.XPATH,
                    value=f"/html/body/div[2]/div[2]/div[2]/div[6]/div[3]/div/a[{next_page}]",
                )
                next_page_button.click()
                time.sleep(2)
                next_page = 2
        except:
            pass
        try:
            if next_page == 5 and not daum_button:
                daum_button = True
                next_page_button = driver.find_element(
                    by=By.XPATH,
                    value=f"/html/body/div[2]/div[2]/div[2]/div[6]/div[3]/div/a[{next_page}]",
                )
                next_page_button.click()
                time.sleep(2)
                next_page = 2
        except:
            pass
    inform = {}
    inform["name"] = restaurant_name
    inform["address"] = address_name
    inform["sort"] = sort
    inform["main_menu"] = ",".join(menu_list)
    inform["mean_price"] = mean_price
    inform["score"] = point
    inform["review_count"] = review_count
    inform["people_give_score"] = cnt
    inform["review_list"] = review_list
    return inform



In [2]:
import time
from selenium import webdriver
import pandas as pd


# 네이버 플레이스에서 음식점에 대한 정보 가져오기

# from image_crawling import total_img_list_func  # 이미지 크롤링 후 리스트 가져오기

if __name__ == "__main__":

    # print(db)
    region_df = pd.read_csv("C:\\Users\\user\\Desktop\\크롤링\\Report.csv")
    region_df = region_df.drop(index=[0, 1, 2], axis=0)
    for region in region_df["법정동"][10:]:

        urls = kakao_restaurant(region, 15)  # 개수는 15의 배수로 적어주세요
        # df = pd.DataFrame(columns=["이름", "분류", "분위기(테마키워드)", "주요 메뉴", "평균 가격", "평점", "리뷰 수"])
        df = pd.DataFrame(
            columns=[
                "name",
                "address",
                "sort",
                "menu",
                "mean_price",
                "score",
                "people_give_score",
                "review_count",
                "review_list",
            ]
        )

        print("region : ", region)
        s = time.time()
        for idx, url in enumerate(urls[:9]):
            print(f"{idx}번째 URL")
            print(url)
            result = kakao_inform_restaurant(url)
            info = {
                "name": result["name"],
                "address": result["address"],
                "sort": result["sort"],
                "menu": result["main_menu"],
                "mean_price": result["mean_price"],
                "score": float(result["score"]),
                "people_give_score": result["people_give_score"],
                "review_count": int(result["review_count"]),
                "review_list": "/".join(result["review_list"]),
            }

            df = pd.concat([df, pd.DataFrame(info, index=[idx])], ignore_index=False)
        df.to_csv(f"{region}.csv", encoding="utf-8-sig")
        print(df)
        e = time.time()
        print("1 region learning time : ", e - s, "s")


region :  옥인동
0번째 URL
http://place.map.kakao.com/1693563906


C:\Users\user\AppData\Local\Temp/ipykernel_27508/1563878121.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


1번째 URL
http://place.map.kakao.com/2134782133
2번째 URL
http://place.map.kakao.com/1427323184
3번째 URL
http://place.map.kakao.com/1269840601
4번째 URL
http://place.map.kakao.com/80159168
5번째 URL
http://place.map.kakao.com/574911495
6번째 URL
http://place.map.kakao.com/1633345942
7번째 URL
http://place.map.kakao.com/27513650
8번째 URL
http://place.map.kakao.com/367404484
      name                        address     sort  \
0  더숲 초소책방       서울 종로구 인왕산로 172 (우)03033      북카페   
1   헤르만의정원                  서울 종로구 옥인동 39       카페   
2      오무사    서울 종로구 필운대로9길 1 1층 (우)03034       술집   
3     옥인23     서울 종로구 필운대로 57 1층 (우)03034    디저트카페   
4   카페알베르게     서울 종로구 필운대로 53-30 (우)03034       카페   
5       안덕  서울 종로구 자하문로17길 18 1층 (우)03035       한식   
6     서촌식당     서울 종로구 자하문로17길 12 (우)03035      한정식   
7       토리       서울 종로구 옥인길 30-3 (우)03034  제과,베이커리   
8      이이엄  서울 종로구 필운대로9길 3 1,2층 (우)03034       카페   

                                       menu mean_price  score  \
0                 아메리카노,에스프레소,카페

C:\Users\user\AppData\Local\Temp/ipykernel_27508/1563878121.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


1번째 URL
http://place.map.kakao.com/14951761
2번째 URL
http://place.map.kakao.com/92217418
3번째 URL
http://place.map.kakao.com/16936957
4번째 URL
http://place.map.kakao.com/704759249
5번째 URL
http://place.map.kakao.com/1940644816
6번째 URL
http://place.map.kakao.com/19119689
7번째 URL
http://place.map.kakao.com/13336632
8번째 URL
http://place.map.kakao.com/1372168402
         name                       address     sort  \
0      토속촌삼계탕      서울 종로구 자하문로5길 5 (우)03041      삼계탕   
1      체부동잔치집  서울 종로구 자하문로1길 16 1층 (우)03041       국수   
2   칸다소바 경복궁점   서울 종로구 자하문로7길 5 1층 (우)03040    일본식라면   
3        대하식당     서울 종로구 자하문로7길 27 (우)03040    육류,고기   
4   서촌뜰애우곱창타운     서울 종로구 자하문로1나길 1 (우)03041    곱창,막창   
5         취천루      서울 종로구 자하문로7길 9 (우)03040     중화요리   
6          마지     서울 종로구 자하문로5길 19 (우)03041       한식   
7  미락치킨 경복궁본점     서울 종로구 자하문로 17-1 (우)03041       치킨   
8    김용현 베이커리    서울 종로구 자하문로 21 1층 (우)03040  제과,베이커리   

                                                menu mean_price  score  \
0       

...

1번째 URL
http://place.map.kakao.com/11016313
2번째 URL
http://place.map.kakao.com/16983115
3번째 URL
http://place.map.kakao.com/212413668
4번째 URL
http://place.map.kakao.com/8612493
5번째 URL
http://place.map.kakao.com/2101093478
6번째 URL
http://place.map.kakao.com/1812014619
7번째 URL
http://place.map.kakao.com/2102346140
8번째 URL
http://place.map.kakao.com/23338568
              name                       address     sort  \
0            서촌계단집  서울 종로구 자하문로1길 15 1층 (우)03041    해물,생선   
1            나무사이로   서울 종로구 사직로8길 21 1층 (우)03169    커피전문점   
2             안주마을      서울 종로구 자하문로1길 3 (우)03041  호프,요리주점   
3             내자상회   서울 종로구 사직로10길 3 1층 (우)03169       카페   
4              할매집    서울 종로구 사직로12길 1-5 (우)03169    족발,보쌈   
5            버거드조선      서울 종로구 사직로10길 7 (우)03169      햄버거   
6       심퍼티쿠시 경복궁점                서울 종로구 내자동 132      와인바   
7              코블러  서울 종로구 사직로12길 16 1층 (우)03169     칵테일바   
8  빚짜X이땔리방앗간 경복궁본점       서울 종로구 사직로 113 (우)03041  호프,요리주점   

                               

C:\Users\user\AppData\Local\Temp/ipykernel_27508/1563878121.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


1번째 URL
http://place.map.kakao.com/1350070025
2번째 URL
http://place.map.kakao.com/735588792
3번째 URL
http://place.map.kakao.com/996804219
4번째 URL
http://place.map.kakao.com/27380134
5번째 URL
http://place.map.kakao.com/826472820
6번째 URL
http://place.map.kakao.com/11943082
7번째 URL
http://place.map.kakao.com/16238023
8번째 URL
http://place.map.kakao.com/26941387
             name                        address      sort  \
0       소문난주문진막국수      부산 동래구 사직로58번길 8 (우)47865        국수   
1        금강만두 직영점  부산 동래구 사직북로5번길 11 1층 (우)47865        분식   
2            옥당분식   부산 동래구 사직북로28번길 101 (우)47852       떡볶이   
3      원조장어구이 사직점       부산 동래구 사직로 3 2층 (우)47875        장어   
4  삼대돼지불고기 2호점손녀집         부산 동래구 사직로 11 (우)47875  불고기,두루치기   
5           초록담미역      부산 동래구 사직로 73 2층 (우)47873        한식   
6         쌍둥이돼지국밥     부산 동래구 아시아드대로 147 (우)47867        국밥   
7           삼천포횟집      부산 동래구 사직로 50 1층 (우)47865         회   
8  더도이종가집돼지국밥 사직점     부산 동래구 아시아드대로 114 (우)47845        국밥   

                      

C:\Users\user\AppData\Local\Temp/ipykernel_27508/1563878121.py:30: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


1번째 URL
http://place.map.kakao.com/886908526
2번째 URL
http://place.map.kakao.com/26844382
3번째 URL
http://place.map.kakao.com/824925847
4번째 URL
http://place.map.kakao.com/15449028
